In [1]:
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import matplotlib.dates as mdates

import warnings
warnings.filterwarnings(action='ignore')

In [44]:
cu = pd.read_csv('EDA_customer.csv')
tr = pd.read_csv('EDA_transaction_last1.csv')

In [45]:
cu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   customer_id         100000 non-null  int64  
 1   gender              100000 non-null  object 
 2   age                 100000 non-null  int64  
 3   join_age            100000 non-null  int64  
 4   device              100000 non-null  object 
 5   device_version      100000 non-null  object 
 6   home_location_lat   100000 non-null  float64
 7   home_location_long  100000 non-null  float64
 8   home_location       100000 non-null  object 
 9   birthdate           100000 non-null  object 
 10  first_join_date     100000 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 8.4+ MB


In [46]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254585 entries, 0 to 1254584
Data columns (total 17 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   created_at           1254585 non-null  object 
 1   customer_id          1254585 non-null  int64  
 2   session_id           1254585 non-null  object 
 3   payment_method       1254585 non-null  object 
 4   payment_status       1254585 non-null  object 
 5   promo_amount         1254585 non-null  int64  
 6   promo_code           1254585 non-null  object 
 7   shipment_fee         1254585 non-null  int64  
 8   shipment_date_limit  1254585 non-null  object 
 9   total_amount         1254585 non-null  int64  
 10  product_id           1254585 non-null  int64  
 11  quantity             1254585 non-null  int64  
 12  item_price           1254585 non-null  int64  
 13  year                 1254585 non-null  int64  
 14  shipment_time        1254585 non-null  object 
 15

### 전처리

In [47]:
tr = tr[tr['payment_status']=='Success']
tr = tr.drop_duplicates(subset='session_id', keep='first')
tr_join = tr[['customer_id','created_at','session_id']]


In [48]:
cu_join = cu[['customer_id','first_join_date']]

In [71]:
df= pd.merge(cu_join, tr_join, on='customer_id', how='left')

In [72]:
df['customer_id'].nunique()

100000

In [73]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['order_date'] = df['created_at'].dt.date
df['order_date'] = pd.to_datetime(df['order_date'])
df['first_join_date'] = pd.to_datetime(df['first_join_date'])

In [74]:
df['session_id'].nunique()

815964

In [75]:
df = df[df['order_date'] != "2016-06-30"]
df = df.drop(columns=['created_at'])

In [76]:
# df.to_csv("customer_status.csv", index=False, encoding="utf-8-sig", mode="w")

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 865721 entries, 0 to 865721
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   customer_id      865721 non-null  int64         
 1   first_join_date  865721 non-null  datetime64[ns]
 2   session_id       815963 non-null  object        
 3   order_date       815963 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 33.0+ MB


In [79]:
df = df.sort_values(by=["customer_id", "order_date"])

df.head()

,customer_id,first_join_date,session_id,order_date
98074,1,2016-11-27,NaN,NaT
368785,2,2021-12-25,NaN,NaT
255100,3,2018-08-25,f03b6825-1e6e-4916-bf71-f94ecbcbaee5,2018-08-25
255101,3,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f,2018-09-22
255102,3,2018-08-25,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7,2018-10-20


In [80]:
# Filter rows where order_date is before September 2016 or NaN
df = df[pd.isnull(df["order_date"]) | (df["order_date"] >= "2016-09-01")]

# Update first_join_date for those who joined before September 2016
df.loc[df["first_join_date"] < "2016-09-01", "first_join_date"] = pd.Timestamp("2016-09-09")

df.head()

,customer_id,first_join_date,session_id,order_date
98074,1,2016-11-27,NaN,NaT
368785,2,2021-12-25,NaN,NaT
255100,3,2018-08-25,f03b6825-1e6e-4916-bf71-f94ecbcbaee5,2018-08-25
255101,3,2018-08-25,36673b2f-940a-42b5-994d-1cce4796b55f,2018-09-22
255102,3,2018-08-25,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7,2018-10-20


In [86]:
df['customer_id'].nunique()

99878

- 측정시점이 first_join_date보다 이전인 경우: "not_join_yet"

- first_join_date에서 첫 4개월 동안:
    - 거래가 있으면: "active_new"
    - 거래가 없으면: "inactive_new"
- 다음 4개월(order_date가 있는지를 기준으로):
    - "active_new"의 상태에서 거래가 있으면: "active_existing"
    - "active_new"의 상태에서 거래가 없으면: "dormant"
    - "inactive_new"의 상태에서 거래가 있으면: "active_existing"
    - "inactive_new"의 상태에서 거래가 없으면: "low_potential_new"
- 이후 4개월:
    - "dormant"의 상태에서 거래가 있으면: "resurrected"
    - "dormant"의 상태에서 거래가 없으면: "dormant"
    - "active_existing"의 상태에서 거래가 없으면: "dormant"
    - "active_existing"의 상태에서 거래가 있으면: "active_existing"
    - "resurrected"의 상태에서 거래가 있으면: "active_existing"
    - "resurrected"의 상태에서 거래가 없으면: "dormant"
    - low_potential_new에서 거래가 있으면 resurrected 
    - low_potential_new에서 거래가 없으면 low_potential_new


- 2016년 9월1일~2022년7월까지의 데이터사용(그 이전거래는 삭제, 그 이전가입자는 2016년 9월1일 가입한것으로 고려. CRM측정이 9월1일부터 생성되었다)
- 4개월 단위로 상태확인하다보니, 첫 측정일은 2017년 1월 1일, 마지막 측정일은 2022년 5월 1일

### feature 생성

In [82]:
# 3. Define the status update functions
def update_status(prev_status, has_transaction):
    if prev_status == "not_join_yet":
        return "active_new" if has_transaction else "inactive_new"
    elif prev_status == "active_new":
        return "active_existing" if has_transaction else "dormant"
    elif prev_status == "inactive_new":
        return "active_existing" if has_transaction else "low_potential_new"
    elif prev_status in ["dormant", "resurrected"]:
        return "active_existing" if has_transaction else "dormant"
    elif prev_status == "active_existing":
        return "active_existing" if has_transaction else "dormant"
    elif prev_status == "low_potential_new":
        return "resurrected" if has_transaction else "low_potential_new"
    else:
        return prev_status

def update_status_v3(prev_status, first_join_date, has_transaction, current_date, start_period):
    if current_date < first_join_date:
        return "not_join_yet"
    elif start_period <= first_join_date <= current_date:
        return "active_new" if has_transaction else "inactive_new"
    else:
        return update_status(prev_status, has_transaction)

In [83]:
# 4. Determine the status for each date range
date_ranges = pd.date_range('2017-01-01', '2022-07-01', freq='4MS')

status_df_new = df[["customer_id"]].drop_duplicates().set_index("customer_id")
status_df_new[date_ranges[0].strftime('%Y-%m-%d')] = "not_join_yet"

for i in range(len(date_ranges)):
    current_date = date_ranges[i]
    start_period = current_date - pd.DateOffset(months=4) if i != 0 else pd.Timestamp("2016-09-01")
    
    for customer_id, group in df.groupby("customer_id"):
        transactions = group[(group["order_date"] > start_period) & (group["order_date"] <= current_date)]
        has_transaction = not transactions.empty
        
        if i == 0:
            prev_status = "not_join_yet"
        else:
            prev_status = status_df_new.loc[customer_id, date_ranges[i-1].strftime('%Y-%m-%d')]
            
        status_df_new.loc[customer_id, current_date.strftime('%Y-%m-%d')] = update_status_v3(prev_status, group["first_join_date"].iloc[0], has_transaction, current_date, start_period)


In [85]:
status_df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99878 entries, 1 to 100000
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   2017-01-01  99878 non-null  object
 1   2017-05-01  99878 non-null  object
 2   2017-09-01  99878 non-null  object
 3   2018-01-01  99878 non-null  object
 4   2018-05-01  99878 non-null  object
 5   2018-09-01  99878 non-null  object
 6   2019-01-01  99878 non-null  object
 7   2019-05-01  99878 non-null  object
 8   2019-09-01  99878 non-null  object
 9   2020-01-01  99878 non-null  object
 10  2020-05-01  99878 non-null  object
 11  2020-09-01  99878 non-null  object
 12  2021-01-01  99878 non-null  object
 13  2021-05-01  99878 non-null  object
 14  2021-09-01  99878 non-null  object
 15  2022-01-01  99878 non-null  object
 16  2022-05-01  99878 non-null  object
dtypes: object(17)
memory usage: 15.7+ MB


In [88]:
status_df_new

,2017-01-01,2017-05-01,2017-09-01,2018-01-01,2018-05-01,2018-09-01,2019-01-01,2019-05-01,2019-09-01,2020-01-01,2020-05-01,2020-09-01,2021-01-01,2021-05-01,2021-09-01,2022-01-01,2022-05-01
customer_id,,,,,,,,,,,,,,,,,
1,inactive_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new,low_potential_new
2,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,inactive_new,low_potential_new
3,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,active_new,active_existing,active_existing,active_existing,active_existing,active_existing,active_existing,active_existing,active_existing,active_existing,active_existing,active_existing
4,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,inactive_new,low_potential_new,low_potential_new,low_potential_new
5,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,inactive_new,low_potential_new
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,inactive_new
99997,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,active_new,dormant,dormant,dormant,dormant,dormant,dormant
99998,not_join_yet,not_join_yet,not_join_yet,not_join_yet,not_join_yet,active_new,dormant,active_existing,active_existing,dormant,active_existing,active_existing,dormant,active_existing,active_existing,dormant,active_existing


In [89]:
cu_join.loc[cu_join["first_join_date"] < "2016-09-01", "first_join_date"] = pd.Timestamp("2016-09-09")

In [93]:
final= pd.merge(cu_join,status_df_new, on='customer_id', how='inner')

In [94]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99878 entries, 0 to 99877
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   customer_id      99878 non-null  int64 
 1   first_join_date  99878 non-null  object
 2   2017-01-01       99878 non-null  object
 3   2017-05-01       99878 non-null  object
 4   2017-09-01       99878 non-null  object
 5   2018-01-01       99878 non-null  object
 6   2018-05-01       99878 non-null  object
 7   2018-09-01       99878 non-null  object
 8   2019-01-01       99878 non-null  object
 9   2019-05-01       99878 non-null  object
 10  2019-09-01       99878 non-null  object
 11  2020-01-01       99878 non-null  object
 12  2020-05-01       99878 non-null  object
 13  2020-09-01       99878 non-null  object
 14  2021-01-01       99878 non-null  object
 15  2021-05-01       99878 non-null  object
 16  2021-09-01       99878 non-null  object
 17  2022-01-01       99878 non-null

In [99]:
final.to_csv('customer_status_NEW.csv',index=False, encoding="utf-8-sig", mode="w")